In [ ]:
from glob import glob
import numpy as np
import pandas as pd
from scipy import stats
import os

from analysis_scripts import second_harm_tools
from analysis_scripts.analyzs_many import get_human

In [ ]:
human_skeleton = glob(os.path.join("..", "data", "human_skeleton", "*"))

In [ ]:
human_skeleton

In [ ]:
defaults =  {"t_range": slice(None, None)}
params = {"Skeletonexpnt006Cell2Small":{
             "t_range": slice(None,5500)},
          
        "Skeletonexpnt006Cell1":{
             "t_range": slice(None,8800)
            } 
         }


results = []
spectrums = []
for exp in human_skeleton:
    param = params.get(exp.split("/")[-1], defaults)
    
    if param.get("skip"):
        continue
    a = get_human(exp, rel_s=0.9, **param)
    if a.last_valid_index() < 0.1:
        continue
    res_human, spectrum = second_harm_tools.analyse_shape(a, NFFT=250, pca=False, smoothing=139)
    res_human.index.name = "time"
    res_human.reset_index(inplace=True)
    res_human["exp"] = exp
    results.append(res_human)
    spectrums.append(spectrum)

res_humans = pd.concat(results)


In [ ]:
len(human_skeleton)

In [ ]:
5./res_humans.dropna().omega.mean()*500

In [ ]:
res_humans["phi_vel_normed_s_c0"] = res_humans.phi_vel_normed_s/(res_humans.C0**2)
res_humans["c2_plus_cm"] = (9/16/np.pi)*res_humans.C2_SIN + (np.pi**2-3)/4/np.pi**2*res_humans.kappa_mean_s

In [ ]:
res = []
for spectrum in spectrums:
    res.append((spectrum[::500,:,0].flatten(), np.sqrt(spectrum[::500,:,1].flatten())))
xx = np.hstack(list(zip(*res))[0])
yy = np.hstack(list(zip(*res))[1])
    

In [ ]:
power, bin_edge, nbins = stats.binned_statistic(xx, yy,
                                                bins=np.arange(0.05,5.,0.1), statistic="mean")
power_std, bin_edge, nbins = stats.binned_statistic(xx, yy,
                                                bins=np.arange(0.05,5.,0.1), statistic="std")

freq = (bin_edge[1:]+bin_edge[:-1])/2


In [ ]:
res_humans.to_hdf(os.path.join("..", "c2_kappa_mean_rotation.h5"), 'humans')
np.savetxt(os.path.join("..", "human_spectrum.csv"),  np.vstack([freq, power, power_std]).T)